In [2]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
sys.path.append("apollo")
from apollo.agent import Agent

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
agent = Agent()

In [ ]:
agent.load('model')

## Conduct a Virtual Screen
To virtually screen a set of compounds, create a CSV file with a column titled `SMILES` that contains the cannonical SMILES string for every chosen compound.
An example of such a file is shown below

|Name|SMILES|
|---|---|
|Dopamine|C1=CC(=C(C=C1CCN)O)O|
|Serotonin|C1=CC2=C(C=C1O)C(=CN2)CCN|
|Epinephrine|CNCC(C1=CC(=C(C=C1)O)O)O|
|Norepinephrine|C1=CC(=C(C=C1C(CN)O)O)O|

Once you've compiled the file, press the upload button to screen your selected compounds

In [ ]:
widgets.interact(agent.screen,file=widgets.FileUpload())